<a href="https://colab.research.google.com/github/AMaheshVardhan/Zeotap-Assignment/blob/main/MaheshVardhan_Akena_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

customers = pd.read_csv('/content/Data/Customers.csv')
transactions = pd.read_csv('/content/Data/Transactions.csv')
products = pd.read_csv('/content/Data/Products.csv')



customers = customers.fillna(0)
transactions = transactions.fillna(0)
products = products.fillna(0)


merged_data = pd.merge(transactions, customers, on='CustomerID', how='left')
merged_data = pd.merge(merged_data, products, on='ProductID', how='left')


customer_features = merged_data.groupby('CustomerID').agg(
    total_spending=('TotalValue', 'sum'),
    total_quantity=('Quantity', 'sum'),
    unique_products=('ProductID', 'nunique')
).reset_index()

customers_data = pd.merge(customers, customer_features, on='CustomerID', how='left')
customers_data = customers_data.fillna(0)


scaler = StandardScaler()
features = ['total_spending', 'total_quantity', 'unique_products']
customers_data[features] = scaler.fit_transform(customers_data[features])

similarity_matrix = cosine_similarity(customers_data[features])

lookalike_recommendations = {}

for i in range(20):
    cust_id = customers_data['CustomerID'].iloc[i]
    similarity_scores = similarity_matrix[i]
    similarity_scores[i]=-1
    top_indices = similarity_scores.argsort()[-3:][::-1]
    recommendations = [(customers_data['CustomerID'].iloc[idx], similarity_scores[idx]) for idx in top_indices]
    lookalike_recommendations[cust_id] = recommendations
lookalike_data = []
for cust_id, recommendations in lookalike_recommendations.items():
    for rec in recommendations:
        lookalike_data.append([cust_id, rec[0], rec[1]])

lookalike_df = pd.DataFrame(lookalike_data, columns=['CustomerID', 'RecommendedCustomerID', 'SimilarityScore'])
lookalike_map = lookalike_df.groupby('CustomerID').apply(lambda x: x[['RecommendedCustomerID', 'SimilarityScore']].values.tolist()).to_dict()
lookalike_final = pd.DataFrame(list(lookalike_map.items()), columns=['CustomerID', 'Recommendations'])
lookalike_final.to_csv('Lookalike.csv', index=False)


<ipython-input-6-0440ccf2c1c1>:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lookalike_map = lookalike_df.groupby('CustomerID').apply(lambda x: x[['RecommendedCustomerID', 'SimilarityScore']].values.tolist()).to_dict()
